In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Journaling `sows`

In [ ]:
# load relevant `ztemp files`

In [ ]:
import glob
from pathlib import Path
from utils import get_pickle, to_list
import pandas as pd
from loguru import logger
import datetime
from typing import Union
from loguru import logger

In [ ]:
pd.set_option('display.max_columns', None)
sow_path = DATAPATH.parent / 'raw'

In [ ]:
from utils import get_archived_sows, async_pf, join_my_df_with_another, pickle_me
import asyncio

df_sowed = get_archived_sows(sow_path, MARKET)

# sort to get latest records on top
df_sows=df_sowed.sort_values('utc_time', ascending=False) \
                    .drop_duplicates(subset=['symbol', 'strike', 'expiry', 'right']) \
                            .set_index('conId')

In [ ]:
# get the dfs
df_pf = asyncio.run(async_pf(PORT)).set_index('conId')


In [ ]:
# join and sort to get latest dfs on top
df = join_my_df_with_another(df_pf, df_sows)
df = df.dropna(subset=['utc_time'])

In [ ]:
JOURNAL_PATH = DATAPATH.parent.parent / 'journal' / ','.join([MARKET.lower(), '_opts.pkl'])

In [ ]:
df_old = get_pickle(JOURNAL_PATH)

In [ ]:
df

In [ ]:
if df_old:
    df_new = pd.concat([df, df_old], axis=0)

In [ ]:
JOURNAL_MASTER_PATH = DATAPATH.parent / 'master' / 'df_journal_opt.pkl'

In [ ]:
pickle_me(df.iloc[0:0], JOURNAL_MASTER_PATH)